In [1]:
import numpy as np 
import pandas as pd 
import os
from keras_callbacks import *
from keras.layers import *
from keras.preprocessing import text, sequence
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import *
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import keras
from keras import backend as K
from random import randint
from keras.engine.topology import Layer
from gensim.models import KeyedVectors
from train_config import targets
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth=True
set_session(tf.Session(config=config))
K.clear_session()

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
D:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
file = ['datasets/train_data_jieba.txt', 'datasets/valid_data_jieba.txt', 'datasets/test_data_jieba.txt']
X_train = []
X_valid = []
X_test = []

for f_name in file:
    with open(f_name,'r',encoding='utf-8') as F:
        if f_name == 'datasets/train_data_jieba.txt':
            line = F.readline()
            while line:
                X_train.append(line)
                line = F.readline()
                
        elif f_name == 'datasets/valid_data_jieba.txt':
            line = F.readline()
            while line:
                X_valid.append(line)
                line = F.readline()
                
        elif f_name == 'datasets/test_data_jieba.txt':
            line = F.readline()
            while line:
                X_test.append(line)
                line = F.readline()

In [3]:
print(len(X_train))
print(len(X_valid))

105000
15000


In [4]:
name = ['location_traffic_convenience',
       'location_distance_from_business_district', 'location_easy_to_find',
       'service_wait_time', 'service_waiters_attitude',
       'service_parking_convenience', 'service_serving_speed', 'price_level',
       'price_cost_effective', 'price_discount', 'environment_decoration',
       'environment_noise', 'environment_space', 'environment_cleaness',
       'dish_portion', 'dish_taste', 'dish_look', 'dish_recommendation',
       'others_overall_experience', 'others_willing_to_consume_again']
train_data = pd.read_csv('datasets/sentiment_analysis_trainingset.csv')
valid_data = pd.read_csv('datasets/sentiment_analysis_validationset.csv')

y_train = []
y_valid = []
for N in name:
    Y_train = train_data[[N]]
    Y_train = np.array(Y_train)+2
    Y_train = to_categorical(Y_train, num_classes=4)
    y_train.append(Y_train)
    
    Y_valid = valid_data[[N]]
    Y_valid = np.array(Y_valid)+2
    Y_valid = to_categorical(Y_valid, num_classes=4)
    y_valid.append(Y_valid)

In [5]:
print(len(y_train[0]))
print(len(y_valid))
print(y_train[3])

105000
20
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [6]:
EMBEDDING_FILE = 'datasets/word2vec_400v'
# 词典大小，embedding矩阵大小
max_features=170000

# 最长句子
maxlen=1024
# 词向量维度
embed_size=400

In [7]:
tok=text.Tokenizer(num_words=max_features,lower=False)
tok.fit_on_texts((X_train + X_valid + X_test))
train = tok.texts_to_sequences(X_train)
valid = tok.texts_to_sequences(X_valid)
test = tok.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(train, maxlen=maxlen, padding='pre')
x_valid = sequence.pad_sequences(valid, maxlen=maxlen, padding='pre')
x_test = sequence.pad_sequences(test, maxlen=maxlen, padding='pre')

In [8]:
embeddings_index = {}
with open(EMBEDDING_FILE,encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [9]:
word_index = tok.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [10]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    return K.mean(2*((precision*recall)/(precision+recall+K.epsilon())))


In [11]:
embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],
                            output_dim=embedding_matrix.shape[1],
                            weights=[embedding_matrix],
                            trainable=False)

In [12]:
def dp_block(input):
    pool = ZeroPadding1D((0, 1))(input)
    pool = MaxPooling1D(pool_size=3, strides=2, padding='valid')(pool)
    conv = Conv1D(filters=128, strides=1, kernel_size=3, padding='same', activation='relu')(pool)  # keep dimension the same
    conv = Conv1D(filters=64, strides=1, kernel_size=3, padding='same', activation='relu')(conv)
    shortcut = Concatenate()([conv, pool])
    return conv

In [17]:
def build_model(embedding_layer, maxlen, targets):
    sent_input = Input((maxlen,), dtype='int32')
    sent_embedded = embedding_layer(sent_input)
    output_list_1 = []
    conv = Conv1D(filters=256, strides=1, kernel_size=3, padding='same', activation='relu')(sent_embedded)
    for target in targets:
    # region embedding
        
        conv_1 = dp_block(conv)

        flat_1 = Flatten(name='flatten_1'+target)(conv_1)
    
        dense = Dense(32, activation='relu', name='Dense_3_'+target)(flat_1)
        output_target = Dense(4, name='output_'+target, activation='softmax')(dense)
        output_list_1.append(output_target)
    
    
    model = Model(sent_input, output_list_1)
    return model

In [14]:
batch_size = 16
epochs = 20

In [18]:
model_name = 'DPCNN'
lr_schedule = generate_learning_rate_schedule(0.0001, 0.05, 20, 0)
checkpoint = generate_check_point(model_name)
early_stopping = generate_early_stopping()
tensorboard = generate_tensorboard(model_name, 'ALL')
callbacks=[lr_schedule, checkpoint, early_stopping, tensorboard]

In [19]:
model = build_model(embedding_layer, maxlen, targets)
model.summary()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[f1])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1024, 400)    68000000    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 1024, 256)    307456      embedding_1[1][0]                
__________________________________________________________________________________________________
zero_padding1d_12 (ZeroPadding1 (None, 1025, 256)    0           conv1d_22[0][0]                  
__________________________________________________________________________________________________
zero_paddi

In [ ]:
model.fit(x_train, 
          y_train, batch_size=batch_size,
          callbacks=callbacks,
          epochs=epochs,
          validation_data=(x_valid, y_valid))

Train on 105000 samples, validate on 15000 samples
Epoch 1/20

Epoch 00001: LearningRateScheduler reducing learning rate to 0.05.
 81584/105000 [======================>.......] - ETA: 9:57 - loss: 115.7021 - output_location_traffic_convenience_loss: 5.5151 - output_location_distance_from_business_district_loss: 3.2822 - output_location_easy_to_find_loss: 3.7465 - output_service_wait_time_loss: 1.9041 - output_service_waiters_attitude_loss: 9.7722 - output_service_parking_convenience_loss: 1.0316 - output_service_serving_speed_loss: 2.5089 - output_price_level_loss: 7.9984 - output_price_cost_effective_loss: 3.7977 - output_price_discount_loss: 12.9368 - output_environment_decoration_loss: 7.8406 - output_environment_noise_loss: 7.7135 - output_environment_space_loss: 6.0821 - output_environment_cleaness_loss: 6.0442 - output_dish_portion_loss: 7.3781 - output_dish_taste_loss: 7.6291 - output_dish_look_loss: 5.9068 - output_dish_recommendation_loss: 3.1116 - output_others_overall_experi